In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
!pip install --upgrade xlrd #colab bug verson xlrd
!pip install geopandas
!pip install pandas fiona shapely pyproj rtree
!pip install contextily
!pip install pygeos
!pip install topojson
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
#!pip install requests py7zr geopandas openpyxl tqdm s3fs PyYAML xlrd
#!pip install git+https://github.com/inseefrlab/cartiflette@80b8a5a28371feb6df31d55bcc2617948a5f9b1a

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 285.5 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 12.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 2.9 MB/s eta 0:00:00


In [2]:
#test_1 = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/5f71ba43-afc8-43a0-b306-dafe29940f9c", sep = ";", encoding = 'utf-8', engine = 'python')
#le lien du dossier data.gouv : 
# https://www.data.gouv.fr/fr/datasets/effectif-de-patients-par-pathologie-sexe-classe-dage-et-territoire-departement-region/
# methode "classique" pour obtenir un dataset a partir d un lien 
# probleme d encodage d ou les parametres sep, encoding, engine
# A FAIRE : COPIER LA BASE DE FACON LOCALE ET S Y REFERER PAR LA SUITE PAR EFFECTIFS.CSV 

In [55]:
test_2 = pd.read_csv("effectifs.csv", sep = ";", encoding = 'utf-8', engine = 'python')
#commande a utiliser une fois qu on a la base 
#copiee en local 
#la base au prealable telechargee sur ordi perso

In [57]:
test_2.head()
test_2.shape

(4057201, 16)

In [5]:
len(test_2['patho_niv1'].unique())

18

In [6]:
print("nombre de categories differentes pour niv2 est :"+str(len(test_2['patho_niv2'].unique())))
print("nombre de categories differentes pour niv3 est :"+str(len(test_2['patho_niv3'].unique())))

nombre de categories differentes pour niv2 est :47
nombre de categories differentes pour niv3 est :60


In [7]:
#potentiellement creer 123 colonnes pour pouvoir faire du clustering 
#comparer les clusters obtenus par les caracteristiques d un inidivu pour voir si ca matche ou pas 


In [8]:
data_gen = test_2

In [58]:
data_gen.head()
data_gen.shape

(1567089, 76)

In [10]:
liste_diff_patho1 = data_gen['patho_niv1'].unique()
liste_diff_patho2 = data_gen['patho_niv2'].unique()
liste_diff_patho3 = data_gen['patho_niv3'].unique()

print(liste_diff_patho1, "\n", liste_diff_patho2, "\n", liste_diff_patho3)

['Traitements du risque vasculaire (hors pathologies)'
 'Traitements psychotropes (hors pathologies)'
 'Séjours en hospitalisation complète pour prise en charge de la Covid-19'
 'Traitement antalgique ou anti-inflammatoire (hors pathologies, traitements, maternité ou hospitalisations)'
 'Maladies respiratoires chroniques (hors mucoviscidose)'
 'Maternité (avec ou sans pathologies)'
 'Pas de pathologies repérées, traitements, maternité, hospitalisations ni traitement antalgique ou anti-inflammatoire'
 'Maladies psychiatriques'
 'Autres affections de longue durée (dont 31 et 32)' 'Cancers'
 'Hospitalisations ponctuelles (avec ou sans pathologies, traitements ou maternité)'
 'Insuffisance rénale chronique terminale' nan 'Diabète'
 'Maladies cardio-neurovasculaires'
 'Maladies du foie ou du pancréas (hors mucoviscidose)'
 'Maladies inflammatoires ou rares ou VIH ou SIDA'
 'Maladies neurologiques ou dégénératives'] 
 ['Traitements hypolipémiants (hors pathologies)' nan
 "Traitements antidép

In [11]:
data_gen.shape

(1567089, 16)

In [12]:
#Creation d une colonne individus
ind = []
for i in range (1,4057202) : 
    ind.append(i)

In [13]:
ind = pd.DataFrame(ind)
ind = ind.set_axis(['ind'], axis='columns')

In [14]:
data_gen_1 = pd.concat([ind, data_gen], axis = 1)

In [15]:
data_gen_1.head()

,ind,annee,patho_niv1,patho_niv2,patho_niv3,top,cla_age_5,sexe,region,dept,ntop,npop,prev,niveau_prioritaire,libelle_classe_age,libelle_sexe,tri
0,1,2020.0,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,1.0,971,290.0,4210.0,6.912,"2,3",de 80 à 84 ans,hommes,51.0
1,2,2020.0,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,3.0,973,40.0,650.0,5.864,"2,3",de 80 à 84 ans,hommes,51.0
2,3,2020.0,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,4.0,999,410.0,5630.0,7.340,"2,3",de 80 à 84 ans,hommes,51.0
3,4,2020.0,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,6.0,999,20.0,350.0,4.913,"2,3",de 80 à 84 ans,hommes,51.0
4,5,2020.0,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,11.0,75,2960.0,20820.0,14.221,"2,3",de 80 à 84 ans,hommes,51.0


In [16]:
#tentative de compréhension de ce que la variable npop représente
b = data_gen['npop'].unique()
b = pd.DataFrame(b)

In [17]:
b = b.dropna()

In [18]:
b = b.set_axis(['test'], axis = 'columns')

In [19]:
b.head()

,test
0,4210.0
1,650.0
2,5630.0
3,350.0
4,20820.0


In [20]:
sum(b['test'])

4734851240.0

In [21]:
#resultat peu concluant

In [22]:
#creation d indicatrices pour futur clustering

In [23]:
type(liste_diff_patho3)

numpy.ndarray

In [24]:
liste_diff_patho3 = list(liste_diff_patho3)

In [25]:
type(liste_diff_patho3)

list

In [44]:
temporaire = []
for i in data_gen['patho_niv3'] : 
    if i == 'Syndrome coronaire aigu' :
        temporaire.append(1)
    else : 
        temporaire.append(0)

data_gen['Syndrome_coronaire_aigu_indicatrice'] = temporaire
data_gen = data_gen.drop(columns = ['Syndrome_coronaire_aigu_indicatrice'])
data_gen.head()

,annee,patho_niv1,patho_niv2,patho_niv3,top,cla_age_5,sexe,region,dept,ntop,npop,prev,niveau_prioritaire,libelle_classe_age,libelle_sexe,tri
0,2020,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,1.0,971,290.0,4210.0,6.912,"2,3",de 80 à 84 ans,hommes,51.0
1,2020,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,3.0,973,40.0,650.0,5.864,"2,3",de 80 à 84 ans,hommes,51.0
2,2020,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,4.0,999,410.0,5630.0,7.340,"2,3",de 80 à 84 ans,hommes,51.0
3,2020,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,6.0,999,20.0,350.0,4.913,"2,3",de 80 à 84 ans,hommes,51.0
4,2020,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,11.0,75,2960.0,20820.0,14.221,"2,3",de 80 à 84 ans,hommes,51.0


In [51]:
def creation_indicatrice (colonne_de_depart, nom_de_maladie) : 
    temporaire = []
    for i in colonne_de_depart :
        if i == f"{nom_de_maladie}" :
            temporaire.append(1)
        else :
            temporaire.append(0)
    data_gen[f"{nom_de_maladie}_ind"] = temporaire


    

In [52]:
for i in liste_diff_patho3 : 
    creation_indicatrice(colonne_de_depart = data_gen['patho_niv3'], nom_de_maladie = i)

In [53]:
data_gen.head()

,annee,patho_niv1,patho_niv2,patho_niv3,top,cla_age_5,sexe,region,dept,ntop,...,Épilepsie_ind,Autres troubles psychiatriques_ind,Démences (dont maladie d'Alzheimer)_ind,Maladie de Parkinson_ind,Myopathie ou myasthénie_ind,Paraplégie_ind,Sclérose en plaques_ind,Transplantation rénale_ind,Suivi de transplantation rénale_ind,None_ind
0,2020,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,1.0,971,290.0,...,0,0,0,0,0,0,0,0,0,0
1,2020,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,3.0,973,40.0,...,0,0,0,0,0,0,0,0,0,0
2,2020,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,4.0,999,410.0,...,0,0,0,0,0,0,0,0,0,0
3,2020,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,6.0,999,20.0,...,0,0,0,0,0,0,0,0,0,0
4,2020,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,11.0,75,2960.0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
data_gen.shape

(1567089, 76)

In [ ]:
#perte d un grand nombre de lignes 
#pb a regler